In [32]:
# Libraries
import tweepy
import csv
import pandas as pd
import json
from datetime import timedelta
from tqdm import tqdm

In [33]:
# 認証情報をJSONファイルから読み込み
json_load = json.load(open('settings.json', 'r'))
API_KEY = json_load["API_KEY"]
API_KEY_SECRET = json_load["API_KEY_SECRET"]
BEARER_TOKEN = json_load["BEARER_TOKEN"]
ACCESS_TOKEN = json_load["ACCESS_TOKEN"]
ACCESS_TOKEN_SECRET = json_load["ACCESS_TOKEN_SECRET"]

# 認証情報を設定
auth = tweepy.OAuthHandler(API_KEY, API_KEY_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

# その他
OUTPUT_FILE = "data/all_tweets.csv"

In [34]:
#latest_tweets = api.user_timeline(count=200)

In [35]:
#latest_tweets[0].created_at + timedelta(hours=+9)

In [36]:
# 全ツイートを入れる空のリストを用意
all_tweets    = []
# 直近の200ツイート分を取得しておく
latest_tweets = api.user_timeline(count=200)
all_tweets.extend(latest_tweets)

# 取得するツイートがなくなるまで続ける
while len(latest_tweets)>0:
    latest_tweets = api.user_timeline(count=200, max_id=all_tweets[-1].id-1)
    all_tweets.extend(latest_tweets)

with open(OUTPUT_FILE, 'w', newline='', encoding='utf_8_sig') as f:
    writer = csv.writer(f)
    writer.writerow(['date', 'tweet_text', '#characters', '#favorited', '#retweeted', 'hasImage'])
    for tweet in tqdm(all_tweets):
        if (tweet.text.startswith('RT')):
            continue # RTはスキップ
        else:
            has_image        = 0 # 画像付きのツイートか
            tweet_characters = tweet.text # ツイートの文字列
            if 'media' in tweet.entities:
                has_image = 1
            if len(tweet.entities['urls']) > 0:
                # urlは、文字数としてカウントしない
                tweet_characters = tweet_characters.strip(tweet.entities['urls'][0]['url']).strip()
            tweet_datetime = tweet.created_at + timedelta(hours=+9)
            tweet_datetime = tweet_datetime.replace(tzinfo=None)
            
            writer.writerow([tweet_datetime ,tweet.text, len(tweet_characters), tweet.favorite_count, tweet.retweet_count, has_image])

100%|██████████| 3230/3230 [00:00<00:00, 202046.20it/s]
